# Benchmarks

## Initialize

In [17]:
import os
import math
import pathlib
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.feather as feather
from tqdm.auto import tqdm
from IPython.display import clear_output

import warnings
from lifelines.utils import CensoringType
from lifelines.utils import concordance_index

In [18]:
import ray
ray.shutdown()

In [19]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: 
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_label = "22_retina_phewas"
project_path = f"{base_path}/results/projects/{project_label}"
figure_path = f"{project_path}/figures"
output_path = f"{project_path}/data"

pathlib.Path(figure_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True)

##### BEGIN ADAPT #####
# second best model
# wandb_name = 'aug++_convnext_s_mlp'
# wandb_id = '8ngm6apd'
# best model
wandb_name = 'aug++_convnext_s_mlp+'
wandb_id = '3p3smraz'
partitions = [0] # [i for i in range(22)]
##### END   ADAPT #####

experiment = wandb_id
experiment_path = f"{output_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

/sc-projects/sc-proj-ukb-cvd


In [20]:
import ray
ray.init(num_cpus=24, include_dashboard=False)#, dashboard_port=24762, dashboard_host="0.0.0.0", include_dashboard=True)#, webui_url="0.0.0.0"))

{'node_ip_address': '10.32.105.6',
 'raylet_ip_address': '10.32.105.6',
 'redis_address': '10.32.105.6:30864',
 'object_store_address': '/tmp/ray/session_2022-05-25_11-18-19_222156_4151528/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-05-25_11-18-19_222156_4151528/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-05-25_11-18-19_222156_4151528',
 'metrics_export_port': 41470,
 'node_id': '71468348c32e1fdc00adde08c4e77b265b564eae4796709b023bcb59'}

In [21]:
import pandas as pd
endpoints = sorted([l.replace('_prevalent', '') for l in list(pd.read_csv('/sc-projects/sc-proj-ukb-cvd/results/projects/22_medical_records/data/220413/frequent_endpoints.csv').endpoint.values)])

In [22]:
covariates = ["age_at_recruitment_f21022_0_0", "sex_f31_0_0",  "ethnic_background_f21000_0_0"]

In [23]:
data_covariates = pd.read_feather(f"{experiment_path}/data_covariates.feather").set_index("eid")[covariates]\
    .assign(age_at_recruitment_f21022_0_0 = lambda x: x.age_at_recruitment_f21022_0_0.astype(np.int32))

In [24]:
data_covariates_ray = ray.put(data_covariates)

In [25]:
variables_to_norm = ["age_at_recruitment_f21022_0_0"] + endpoints
len(variables_to_norm)

499

In [26]:
in_path = pathlib.Path(f"{experiment_path}/loghs")
in_path.mkdir(parents=True, exist_ok=True)

out_path = f"{experiment_path}/coxph/input"
pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)

In [27]:
models = [f.name for f in in_path.iterdir() if f.is_dir() and "ipynb_checkpoints" not in str(f)]
print(models)

['ImageTraining_[]_ConvNeXt_MLPHead']


In [28]:
from sklearn.preprocessing import StandardScaler
import pickle
import zstandard

def read_merge_data(fp_in, split, data_covariates):
    temp = pd.read_feather(f"{fp_in}/{split}.feather").set_index("eid")
    temp = temp.merge(data_covariates, left_index=True, right_index=True, how="left")
    return temp   
    
def save_pickle(data, data_path):
    with open(data_path, "wb") as fh:
        cctx = zstandard.ZstdCompressor()
        with cctx.stream_writer(fh) as compressor:
            compressor.write(pickle.dumps(data, protocol=pickle.HIGHEST_PROTOCOL))
    
@ray.remote
def norm_variables(data_covariates, model, partition, variables):

    fp_in = f"{in_path}/{model}/{partition}"
    fp_out = f"{out_path}/{model}/{partition}"
    
    if pathlib.Path(fp_in).is_dir():
        if not pathlib.Path(fp_out).is_dir():
            pathlib.Path(fp_out).mkdir(parents=True, exist_ok=True)
            for split in ["train", "valid", 'test']: # "test_left", 'test_right'
                temp = read_merge_data(fp_in, split, data_covariates)
                if split=="train": 
                    scaler = StandardScaler(with_mean=True, with_std=True, copy=True).fit(temp[variables].values)
                    save_pickle(scaler, f"{fp_out}/scaler.p")
                temp[variables] = scaler.transform(temp[variables].values)
                temp.reset_index(drop=False).to_feather(f"{fp_out}/{split}.feather")
    return True

In [29]:
def norm_logh_and_extra(data_covariates_ray, variables):
    progress = []
    for model in tqdm(models):
        for partition in [p for p in partitions]: 
            progress.append(norm_variables.remote(data_covariates, model, partition, variables))
    [ray.get(s) for s in tqdm(progress)]

In [30]:
norm_logh_and_extra(data_covariates_ray, variables_to_norm)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [31]:
1+1

2